### Install the libraries

In [ ]:
!pip install langsmith openinference-instrumentation-crewai openinference-instrumentation-groq -q

### Run CrewAI Agent

In [7]:
from crewai import Agent, Task, Crew, LLM, Process
from langsmith.integrations.otel import OtelSpanProcessor
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from openinference.instrumentation.crewai import CrewAIInstrumentor
from openinference.instrumentation.groq import GroqInstrumentor

load_dotenv()

tracer_provider = trace.get_tracer_provider()
if not isinstance(tracer_provider, TracerProvider):
    tracer_provider = TracerProvider()
    trace.set_tracer_provider(tracer_provider)

tracer_provider.add_span_processor(OtelSpanProcessor())

CrewAIInstrumentor().instrument()
GroqInstrumentor().instrument()

llm = LLM(model = "groq/llama-3.1-8b-instant", 
        temperature = 0.2,
        max_completion_tokens = 256,
        top_p = 0.9
    )

llm = LLM(model = "groq/llama-3.1-8b-instant", 
        temperature = 0.2,
        max_completion_tokens = 256,
        top_p = 0.9
    )

planner = Agent(
    role = "Content Planner", 
    goal = "Plan engaging and factually accurate content on {topic}", 
    backstory = "You're working on planning a blog article about the: {topic}"
                "You collect information that helps the audience learn something"
                "and make informed decisions."
                "Your work is the basis for the content writer to write an article on this topic.", 
    llm = llm, 
    allow_delegation = False, 
    verbose = True
)

writer = Agent(
    role = "Content Writer",
    goal = "Write a factually accurate opinion piece about the topic: {topic}",
    backstory = "You are working on writing a new opinion piece about the topic: {topic}."
                "You base your writing on the work of the content planner, who provides an outline"
                "You also provide objective and insights and back them up with the information"
                "provided by the content planner",
    llm = llm, 
    allow_delegation = False, 
    verbose = True
)

editor = Agent(
    role = "Editor", 
    goal = "Edit the given blog post with a proper writing style.",
    backstory = "You an editor who receives a blog post from the content writer."
                "Your goal is to ensure that it follows journalistic best practices.",
    llm = llm, 
    allow_delegation = False, 
    verbose = True
)   

plan = Task(
    description = (
        "1. Prioritize the latest trends and news on the {topic}.\n"
        "2. Identify the target audience based on their interests.\n"
        "3. Develop a detailed content outline including an introduction, and key points.\n"
        "4. Include SEO keywords and relevant data sources.\n"
    ),
    expected_output = "A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources.\n",
    agent = planner
)

write = Task(
    description = (
        "1. Use the content plan to craft a blog post on the {topic}.\n"
        "2. Incoporate SEO keywords naturally.\n"
        "3. Ensure the post is properly structured with an introduction, insightful body, and a summarising conclusion.\n"
        "4. Proofread for grammatical errors.\n"
    ),
    expected_output = "A well-written blog post in markdown format, ready for publication.", 
    agent = writer
)

edit = Task(
    description = (
        "Proofread the given blog post for grammatical errors and alignment with the core-opinion."
    ),
    expected_output = "A well-written blog post in markdown format, ready for publication.",
    agent = editor
)

crew = Crew(
    agents = [planner, writer, editor],
    tasks = [plan, write, edit],
    verbose = True, 
    tracing = True
)   

try: 
    result = crew.kickoff(inputs = {"topic": "Autonomous Enterprises"})
    print("Result:", result)
except Exception as e: 
    print(f"An error occured: {e}")

Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 227eaa75-ce25-4b73-a1e0-8d00956063b9                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends and news on the Autonomous Enterprises.                                  │
│  2. Identify the target audience based on their interests.                                                      │
│  3. Develop a detailed content outline including an introduction, and key points.                               │
│  4. Include SEO keywords and relevant data sources.                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Failed to export span batch code: 409, reason: 
payloads already received

Output()

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Final Answer                                                                                                   │
│                                                                                                                 │
│  **Autonomous Enterprises: Revolutionizing Business Operations**                                                │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Introduction**                                                                                               │
│                                                                                                                 │
│  In today's fast-paced business landscape, companies are constantly seeking innovative ways to stay ahead of    │
│  the competition. One approach that has gained significant attention in recent years is Autonomous              │
│  Enterprises. This concept involves leveraging technology, particularly Artificial Intelligence (AI) and        │
│  Machine Learning (ML), to automate business operations, making decisions, and driving growth. In this          │
│  article, we will delve into the world of Autonomous Enterprises, exploring its benefits, challenges, and the   │
│  latest trends and news in this exciting field.                                                                 │
│                                                                                                                 │
│  **What are Autonomous Enterprises?**                                                                           │
│                                                                                                                 │
│  Autonomous Enterprises are businesses that utilize AI and ML to automate various aspects of their operations,  │
│  such as decision-making, customer service, and supply chain management. This approach enables companies to     │
│  streamline processes, reduce costs, and enhance efficiency, ultimately leading to improved customer            │
│  experiences and increased revenue.                                                                             │
│                                                                                                                 │
│  **Benefits of Autonomous Enterprises**                                                                         │
│                                                                                                                 │
│  1. **Increased Efficiency:** Autonomous Enterprises can automate repetitive tasks, freeing up human resources  │
│  to focus on high-value activities that drive growth and innovation.                                            │
│  2. **Improved Decision-Making:** AI and ML algorithms can analyze vast amounts of data, providing businesses   │
│  with accurate and timely insights to inform strategic decisions.                                               │
│  3. **Enhanced Customer Experience:** Autonomous Enterprises can offer personalized services                    │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 301e2483-24c7-465a-92d5-f9ce5e50e054                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 227eaa75-ce25-4b73-a1e0-8d00956063b9                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Final Answer                                                                                     │
│                                                                                                                 │
│  **Autonomous Enterprises: Revolutionizing Business Operations**                                                │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Introduction**                                                                                               │
│                                                                                                                 │
│  In today's fast-paced business landscape, companies are constantly seeking innovative ways to stay ahead of    │
│  the competition. One approach that has gained significant attention in recent years is Autonomous              │
│  Enterprises. This concept involves leveraging technology, particularly Artificial Intelligence (AI) and        │
│  Machine Learning (ML), to automate business operations, making decisions, and driving growth. In this          │
│  article, we will delve into the world of Autonomous Enterprises, exploring its benefits, challenges, and the   │
│  latest trends and news in this exciting field.                                                                 │
│                                                                                                                 │
│  **What are Autonomous Enterprises?**                                                                           │
│                                                                                                                 │
│  Autonomous Enterprises are businesses that utilize AI and ML to automate various aspects of their operations,  │
│  such as decision-making, customer service, and supply chain management. This approach enables companies to     │
│  streamline processes, reduce costs, and enhance efficiency, ultimately leading to improved customer            │
│  experiences and increased revenue.                                                                             │
│                                                                                                                 │
│  **Benefits of Autonomous Enterprises**                                                                         │
│                                                                                                                 │
│  1. **Increased Efficiency:** Autonomous Enterprises can automate repetitive tasks, freeing up human resources  │
│  to focus on high-value activities that drive growth and innovation.                                            │
│  2. **Improved Decision-Making:** AI and ML algorithms can analyze vast amounts of data, providing businesses   │
│  with accurate and timely insights to inform strategic decisions.                                               │
│  3. **Enhanced Customer Experience:** Autonomous Enter

╭─────────────────────────────────────────── Trace Batch Finalization ────────────────────────────────────────────╮
│ ✅ Trace batch finalized with session ID: 19f2367a-5cd4-4a84-be07-7f803eaa078d                                  │
│                                                                                                                 │
│ 🔗 View here:                                                                                                   │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/19f2367a-5cd4-4a84-be07-7f803eaa078d?access_code=TRA │
│ CE-7b707208dd                                                                                                   │
│ 🔑 Access Code: TRACE-7b707208dd                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Result: Final Answer

**Autonomous Enterprises: Revolutionizing Business Operations**

---

**Introduction**

In today's fast-paced business landscape, companies are constantly seeking innovative ways to stay ahead of the competition. One approach that has gained significant attention in recent years is Autonomous Enterprises. This concept involves leveraging technology, particularly Artificial Intelligence (AI) and Machine Learning (ML), to automate business operations, making decisions, and driving growth. In this article, we will delve into the world of Autonomous Enterprises, exploring its benefits, challenges, and the latest trends and news in this exciting field.

**What are Autonomous Enterprises?**

Autonomous Enterprises are businesses that utilize AI and ML to automate various aspects of their operations, such as decision-making, customer service, and supply chain management. This approach enables companies to streamline processes, reduce costs, and enhance efficiency, ultima